In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid
import warnings
warnings.filterwarnings('ignore')

# Load the dataset (replace 'your_dataset.csv' with your actual dataset path)
df = pd.read_csv('AEP_hourly.csv')

# Convert Datetime column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)
df.sort_index(inplace=True)

# Define the parameter grid for ARIMA and ANN hyperparameters
arima_params = {'p': [3,4, 5, 7], 'd': [0,1,4], 'q': [0,1,3]}
ann_params = {'layers': [(64,), (128, 64), (128, 64, 32)], 'epochs': [50, 100], 'batch_size': [16, 32],'activation':['relu','tanh'],'optimizer':['adam','sgd']}

best_r2_score = -float('inf')
best_params = {}

# Perform grid search over the parameter grid
for arima_config in ParameterGrid(arima_params):
    p, d, q = arima_config['p'], arima_config['d'], arima_config['q']
    
    # Fit ARIMA model to the dataset
    arima_order = (p, d, q)
    model_arima = ARIMA(df['AEP_MW'], order=arima_order)
    arima_results = model_arima.fit()

    for ann_config in ParameterGrid(ann_params):
        layers, epochs, batch_size = ann_config['layers'], ann_config['epochs'], ann_config['batch_size']
        
        # Forecast using ARIMA model for future time points
        future_steps = 24  # Number of future steps to forecast
        arima_forecast = arima_results.forecast(steps=future_steps)

        # Convert arima_forecast to a NumPy array
        arima_forecast_np = arima_forecast.values

        # Calculate residuals
        actual_values = df['AEP_MW'].values
        residuals = actual_values[-future_steps:] - arima_forecast_np

        # Use ARIMA forecast as input features (X) and residuals as target (y) for ANN
        X = arima_forecast_np.reshape(-1, 1)  # Reshape to 2D array for compatibility with scaler
        y = residuals.reshape(-1, 1)  # Reshape to 2D array for compatibility with scaler

        # Normalize input features (X) and target (y) using MinMaxScaler
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X)
        y_scaled = scaler_y.fit_transform(y)

        # Define and train the ANN model
        model_ann = Sequential([Dense(units=layer, activation='relu') for layer in layers] + [Dense(1)])  # Dynamic architecture
        model_ann.compile(optimizer='adam', loss='mean_squared_error')
        model_ann.fit(X_scaled, y_scaled, epochs=epochs, batch_size=batch_size, verbose=0)  # Train the model

        # Make predictions using the trained ANN
        ann_predictions_scaled = model_ann.predict(X_scaled)

        # Inverse scaling to get final predictions (residuals)
        final_residuals = scaler_y.inverse_transform(ann_predictions_scaled)

        # Generate corrected forecasts by adding final_residuals to arima_forecast_np
        corrected_forecasts = arima_forecast_np + final_residuals.flatten()

        # Calculate R-squared (coefficient of determination)
        r2 = r2_score(actual_values[-future_steps:], corrected_forecasts)

        # Update best R-squared score and parameters if current model is better
        if r2 > best_r2_score:
            best_r2_score = r2
            best_params = {'ARIMA': arima_config, 'ANN': ann_config}

print(f"Best R-squared (R2) Score: {best_r2_score:.4f}")
print("Best Parameters:")
print(best_params)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━